# [문제] Managing History
-trim messages()
-RunnablePassthrough
-itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation)
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [ ]:
from operator import itemgetter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.chat_history import ( 
  InMemoryChatMessageHistory, 
  BaseChatMessageHistory
)

# 2) .env 파일에서 환경변수 읽어오기
load_dotenv()

# 3) 세션별 대화 히스토리를 저장할 임시 메모리 저장소
store = {}


# 4) 함수 정의: 세션 ID에 따라 대화 히스토리 반환

def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


# 5) prompt template 정의
messages=[
    ('system','''당신은 이력서 작성 컨설턴트입니다.
사용자가 제공한 정보를 바탕으로 자연스럽고 완성도 높은 이력서를 작성합니다.'''),
    ('placeholder', '{chat_history}'),
# ('user','이름:{name}\n경력:{experience}\n기술:{skills}')
    ('user','{query}') #사용자 질문을 하나의 변수로 넣어줘야함.
]


prompt = ChatPromptTemplate.from_messages(messages)


# 6) ChatOpenAI 인스턴스 생성: 모델 생성
llm=ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


# 7) trim 설정
trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)


# 8) chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
    )

# 9) chain에 대화 히스토리 기능을 래핑해서 추가 
with_message_history = RunnableWithMessageHistory( 
    chain, 
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)


# 10) chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>>')

    if query.upper() == 'S':
        break




    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id':'ilovejibong'}}
    ) 

    print('\n'+'='*100+'\n')

True

In [44]:
for message in get_history('ilovejibong').messages:
    print(f'[{message.type.upper()}]: {message.content}\n')

[HUMAN]: 내 이름은 이히빈, 나이는 26살 지원분야는 멀티미디어컨텐츠전문가야 보유능력은 포토샵 1급, 일러스트 2급, 프리미어프로, 애프터이펙트 중급이상 가능 번외로 중학생때 미국 유학을 갔다와서 영어프리토킹 가능해

[AI]: **이히빈 이력서**

---

**개인 정보**

- **이름:** 이히빈
- **나이:** 26세
- **연락처:** [연락처 입력]
- **이메일:** [이메일 입력]

---

**지원 분야**

- **멀티미디어 컨텐츠 전문가**

---

**보유 능력**

- **그래픽 디자인 및 영상 편집**
  - 어도비 포토샵 1급
  - 어도비 일러스트레이터 2급
  - 어도비 프리미어 프로 중급 이상
  - 어도비 애프터 이펙트 중급 이상

- **언어 능력**
  - 영어 프리토킹 가능 (중학생 시절 미국 유학 경험)

---

**경력 및 활동**

- [회사명/기관명], [직위], [근무기간]  
  - [담당 업무 및 성과 기술]
  
- [프로젝트명], [역할], [기간]  
  - [프로젝트 설명 및 기여도]

---

**학력**

- [대학교명], [전공], [졸업 연도]
  - [유관 과목 및 성적]

- [고등학교명], [졸업 연도]

---

**추가 정보**

- 멀티미디어 콘테스트 [수상 경력] (년, 대회 이름)
- [온라인 포트폴리오 링크 또는 성과물 링크]

---

이력서에 포함할 추가 정보를 제공해주시면 보다 완성된 문서를 작성할 수 있습니다. 궁금한 점이 있거나 수정할 사항이 있다면 언제든지 말씀해 주세요!

[HUMAN]: 내 이름이 뭐라고?

[AI]: 죄송하지만, 제공된 정보에서는 귀하의 이름을 알 수 없습니다. 이력서 작성을 원하시면 이름과 관련된 세부 정보를 제공해 주시기 바랍니다.



In [48]:
%whos


Variable                         Type                          Data/Info
------------------------------------------------------------------------
BaseChatMessageHistory           ABCMeta                       <class 'langchain_core.ch<...>.BaseChatMessageHistory'>
ChatOpenAI                       ModelMetaclass                <class 'langchain_openai.<...>_models.base.ChatOpenAI'>
ChatPromptTemplate               ModelMetaclass                <class 'langchain_core.pr<...>chat.ChatPromptTemplate'>
InMemoryChatMessageHistory       ModelMetaclass                <class 'langchain_core.ch<...>emoryChatMessageHistory'>
MessagesPlaceholder              ModelMetaclass                <class 'langchain_core.pr<...>hat.MessagesPlaceholder'>
RunnablePassthrough              ModelMetaclass                <class 'langchain_core.ru<...>ugh.RunnablePassthrough'>
RunnableWithMessageHistory       ModelMetaclass                <class 'langchain_core.ru<...>nableWithMessageHistory'>
StreamingStdOutCallba

In [46]:
get_history('ilovejibong')


InMemoryChatMessageHistory(messages=[HumanMessage(content='내 이름은 이히빈, 나이는 26살 지원분야는 멀티미디어컨텐츠전문가야 보유능력은 포토샵 1급, 일러스트 2급, 프리미어프로, 애프터이펙트 중급이상 가능 번외로 중학생때 미국 유학을 갔다와서 영어프리토킹 가능해', additional_kwargs={}, response_metadata={}), AIMessage(content='**이히빈 이력서**\n\n---\n\n**개인 정보**\n\n- **이름:** 이히빈\n- **나이:** 26세\n- **연락처:** [연락처 입력]\n- **이메일:** [이메일 입력]\n\n---\n\n**지원 분야**\n\n- **멀티미디어 컨텐츠 전문가**\n\n---\n\n**보유 능력**\n\n- **그래픽 디자인 및 영상 편집**\n  - 어도비 포토샵 1급\n  - 어도비 일러스트레이터 2급\n  - 어도비 프리미어 프로 중급 이상\n  - 어도비 애프터 이펙트 중급 이상\n\n- **언어 능력**\n  - 영어 프리토킹 가능 (중학생 시절 미국 유학 경험)\n\n---\n\n**경력 및 활동**\n\n- [회사명/기관명], [직위], [근무기간]  \n  - [담당 업무 및 성과 기술]\n  \n- [프로젝트명], [역할], [기간]  \n  - [프로젝트 설명 및 기여도]\n\n---\n\n**학력**\n\n- [대학교명], [전공], [졸업 연도]\n  - [유관 과목 및 성적]\n\n- [고등학교명], [졸업 연도]\n\n---\n\n**추가 정보**\n\n- 멀티미디어 콘테스트 [수상 경력] (년, 대회 이름)\n- [온라인 포트폴리오 링크 또는 성과물 링크]\n\n---\n\n이력서에 포함할 추가 정보를 제공해주시면 보다 완성된 문서를 작성할 수 있습니다. 궁금한 점이 있거나 수정할 사항이 있다면 언제든지 말씀해 주세요!', additional_kwargs={}, response_metadata={'finish